# Gemini with Pydantic utilising robust output

In [ ]:
from google import genai
import pydantic

client = genai.Client()

response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents="List a few asian soups vegeterian recipies, a yummy description and list the ingredients"
)

print(response.text)

Here are a few popular Asian soup recipes, complete with a yummy description and ingredients list to get your taste buds tingling!

---

### 1. Vietnamese Pho Bo (Beef Noodle Soup)

**A Yummy Description:**
Immerse yourself in the soul-soothing embrace of Pho Bo, Vietnam's iconic national dish. Imagine a deeply aromatic, crystal-clear beef broth, slow-simmered for hours with star anise, cinnamon, ginger, and cloves, infusing it with an intoxicating fragrance that fills your kitchen. This fragrant elixir cradles silky flat rice noodles and tender slices of rare beef (cooked to perfection by the hot broth itself!), crowned with a vibrant bouquet of fresh herbs like cilantro, basil, and mint, along with crisp bean sprouts and a squeeze of lime. Each spoonful is a harmonious blend of savory, sweet, herbaceous, and umami – pure comfort in a bowl that warms you from the inside out.

**Ingredients:**

*   **For the Broth:**
    *   2-3 lbs beef marrow bones, knuckle bones, or oxtail
    *   1

In [ ]:
from pydantic import BaseModel

class Recipe(BaseModel):
    recipe_name: str
    description : str
    ingredients: list[str]

client = genai.Client()

response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents="List a few asian soups vegeterian recipies, a yummy description and list the ingredients",
    config= {"response_mime_type": "application/json", "response_schema": list[Recipe]}
)

print(response.text)

[
  {
    "recipe_name": "Classic Vegetarian Miso Soup",
    "description": "A comforting and savory Japanese soup, light yet deeply flavorful, made with fermented soybean paste and seaweed, perfect for a quick, healthy meal.",
    "ingredients": [
      "4 cups dashi (vegetarian, made with kombu and shiitake)",
      "4 tablespoons white miso paste",
      "8 oz silken tofu, cubed",
      "2 sheets nori, thinly sliced",
      "2 green onions, sliced"
    ]
  },
  {
    "recipe_name": "Spicy Vegetarian Hot and Sour Soup",
    "description": "A delightful blend of spicy and tangy flavors, this classic Chinese soup is packed with earthy mushrooms, crisp bamboo shoots, and a warming kick.",
    "ingredients": [
      "6 cups vegetable broth",
      "1 cup shiitake mushrooms, sliced",
      "1/2 cup bamboo shoots, julienned",
      "1/4 cup wood ear mushrooms, sliced",
      "2 tablespoons rice vinegar",
      "1 tablespoon soy sauce",
      "1 teaspoon sesame oil",
      "1/2 teaspoon whi

In [6]:
recipes = response.parsed

In [7]:
len(recipes)

3

In [8]:
recipes[0].recipe_name

'Classic Vegetarian Miso Soup'

In [9]:
recipes[0].ingredients

['4 cups dashi (vegetarian, made with kombu and shiitake)',
 '4 tablespoons white miso paste',
 '8 oz silken tofu, cubed',
 '2 sheets nori, thinly sliced',
 '2 green onions, sliced']

## Simulate house prices

In [ ]:
from typing import Literal

class Home(BaseModel):
    price : int
    monthly_fee: int
    living_area: float
    number_rooms: int
    type: Literal["apppartment", "house"]
    address : str

response = client.models.generate_content(
    model= "gemini-2.5-pro",
    contents="""
    List 50 apartments and houses in Sweden with their monthly fee, price
    living_area, number_rooms, address, type if it is aprtment or house.
    All currencies in SEK. Data shoudl be simulation of Swedish housing.
    """,
    
    config={"response_mime_type" : "application/json", "response_schema":list[Home]}

)

In [13]:
homes = response.parsed
len(homes)

50

In [14]:
homes[0].price, homes[0].address

(5250000, 'Hornsgatan 78, 118 21 Stockholm')

In [26]:
homes[0].__dict__

{'price': 5250000,
 'monthly_fee': 3150,
 'living_area': 62.5,
 'number_rooms': 2,
 'type': 'apppartment',
 'address': 'Hornsgatan 78, 118 21 Stockholm'}

In [29]:
import pandas as pd

df = pd.DataFrame([home.__dict__ for home in homes])
df.head()

,price,monthly_fee,living_area,number_rooms,type,address
0,5250000,3150,62.5,2,apppartment,"Hornsgatan 78, 118 21 Stockholm"
1,11500000,4200,175.0,6,house,"Villavägen 14, 181 55 Lidingö"
2,3800000,4200,85.0,3,apppartment,"Linnégatan 33, 413 04 Göteborg"
3,6800000,3800,140.0,5,house,"Björkvägen 9, 436 39 Askim"
4,2950000,4800,78.5,3,apppartment,"Ribersborgsvägen 5, 217 53 Malmö"


In [33]:
cheap_houses = df.query("price < 5000000 and type == 'house'" )
cheap_houses

,price,monthly_fee,living_area,number_rooms,type,address
5,4500000,3100,125.0,4,house,"Strandgatan 22, 237 35 Bjärred"
21,3600000,2800,115.0,4,house,"Pilfinksvägen 7, 906 51 Umeå"
26,4250000,3000,130.0,5,house,"Vallmovägen 11, 224 56 Lund"
33,2900000,2500,110.0,4,house,"Björkstigen 5, 906 21 Umeå"


In [34]:
cheap_houses.to_csv("cheap_house.csv", index=False)

In [51]:
from dotenv import load_dotenv
from google import genai
from pydantic import BaseModel, Field
from datetime import datetime

load_dotenv()

class Book(BaseModel):
    id: int
    title : str
    author : str
    publication_year : int = Field(ge= 500, le=datetime.now().year)

class Library(BaseModel):
    name: str
    books: list[Book]

client = genai.Client()

response = client.models.generate_content(
    model= "gemini-2.5-pro",
    contents="""
    You are a helpful asisstant. I need you to create a JSON object representing a library.
    The library's name is 'Biblioteket i Stan' and have the fields; name and books. Book contains a list of books.
    Each book should have an id, title, author, publication_year.
    Give me 10 books.
    You MUST only output a valid JSON fromat
    """,
    config= {"response_mime_type": "application/json", "response_schema": Library}
    )

library = Library.model_validate_json(response.text)
library

Library(name='Biblioteket i Stan', books=[Book(id=1, title='The Lord of the Rings', author='J.R.R. Tolkien', publication_year=1954), Book(id=2, title='Pride and Prejudice', author='Jane Austen', publication_year=1813), Book(id=3, title='To Kill a Mockingbird', author='Harper Lee', publication_year=1960), Book(id=4, title='1984', author='George Orwell', publication_year=1949), Book(id=5, title='The Great Gatsby', author='F. Scott Fitzgerald', publication_year=1925), Book(id=6, title='Moby Dick', author='Herman Melville', publication_year=1851), Book(id=7, title='War and Peace', author='Leo Tolstoy', publication_year=1869), Book(id=8, title='The Catcher in the Rye', author='J.D. Salinger', publication_year=1951), Book(id=9, title='The Hobbit', author='J.R.R. Tolkien', publication_year=1937), Book(id=10, title='Brave New World', author='Aldous Huxley', publication_year=1932)])

In [72]:
with open("library.json", "w") as json_file:
    json_file.write(library.model_dump_json(indent=4))
